**This notebook contains all the processes used to define reps, rows, and zones for Digital Acre 2022. Additionally, it has all the methods used to number reps, rows, zones, and plants.**

In [12]:
import arcpy
from arcpy import env
import pandas as pd

arcpy.env.overwriteOutput = True

# Define Reps Usings Stakes + Numbering

### Merge Stakes

In [2]:
# Fetch all fc layers on their coresponding gdb 
R1_stakes = r"C:\Users\abe_lizbethp\Box\2022_Digital_Acre\Tags\Rep1_stakes_plants_points\Rep1_stakes_plants_points.gdb\Rep1_stakes"
R2_stakes = r"C:\Users\abe_lizbethp\Box\2022_Digital_Acre\Tags\Rep2_stakes_plants_points\Rep2_stakes_plants_points.gdb\Rep2_stakes"
R3_stakes = r"C:\Users\abe_lizbethp\Box\2022_Digital_Acre\Tags\Rep3_stakes_plants_points\Rep3_stakes_plants_points.gdb\Rep3_stakes"
R4_stakes = r"C:\Users\abe_lizbethp\Box\2022_Digital_Acre\Tags\Rep4_stakes_plants_points\Rep4_stakes_plants_points.gdb\Rep4_stakes"
R5_9stakes = r"C:\Users\abe_lizbethp\Box\2022_Digital_Acre\Tags\Rep_5_9_stakes\Rep_5_9_stakes.gdb\rep_5__stakes"

#Local Variables
in_data = [R1_stakes,R2_stakes,R3_stakes,R4_stakes,R5_9stakes]
out_data = "Merge_Stakes"
#Merge all stakes feature class into a single feature class 
arcpy.Merge_management(in_data, out_data)

<Result 'C:\\Users\\abe_lizbethp\\OneDrive - Iowa State University\\Documents\\ArcGIS\\Projects\\Digital_Acre_SOPs\\Digital_Acre_SOPs.gdb\\Merge_Stakes'>

### Convert Stakes points into Rep polygons

In [5]:
# Set local variables
in_point_features = out_data     #Merged Stakes/ output from last geoprocess
out_feature_class = "R1_R9_Polygons"
aggregation_distance = "15 meters"

# Execute Aggregate Points
arcpy.AggregatePoints_cartography(in_point_features, out_feature_class, aggregation_distance)

<Result 'C:\\Users\\abe_lizbethp\\OneDrive - Iowa State University\\Documents\\ArcGIS\\Projects\\Digital_Acre_SOPs\\Digital_Acre_SOPs.gdb\\R1_R9_Polygons'>

### Number Reps

In [6]:
#Calculate center of rep polygon longitude and use longitude to number polygons 
arcpy.env.outputCoordinateSystem = arcpy.Describe("R1_R9_Polygons").spatialReference

# Set local variables
in_features = "R1_R9_Polygons"
arcpy.management.CalculateGeometryAttributes(in_features, [["Rep_Longitude", "CENTROID_X"]])

#Sort polygons by longitude (ascending) and assign numbering 
#Set local variables 
in_dataset = "R1_R9_Polygons"
out_dataset  = "R1_R9_w_Numb"

#Process: Sort (Sort) (management)
arcpy.management.Sort(in_dataset, out_dataset, sort_field=[["Rep_Longitude", "ASCENDING"]], spatial_sort_method="UR")

# Set local variables
inTable = "R1_R9_w_Numb"
fieldName = "Rep_Numb"
expression = "autoIncrement()"

codeblock = """
rec = 0
def autoIncrement():
    global rec
    pStart    = 1
    pInterval = 1
    if (rec == 0):
        rec = pStart
    else:
        rec += pInterval
    return rec"""
 
# Run AddField
arcpy.management.AddField(inTable, fieldName, "SHORT")
 
# Run CalculateField 
arcpy.management.CalculateField(inTable, fieldName, expression, "PYTHON3", 
                                codeblock)

<Result 'R1_R9_w_Numb'>

# Divide Reps into Rows + Numbering

### Divide Reps into Rows

In [7]:
#Subdivide rep in 48 equal parts
in_polygons = "R1_R9_w_Numb"
out_feature_class = "Rows_polygon"
arcpy.management.SubdividePolygon(in_polygons, out_feature_class, 
                                  method="NUMBER_OF_EQUAL_PARTS", num_areas=48, target_area="", target_width="", 
                                  split_angle=0, subdivision_type="STRIPS")

<Result 'C:\\Users\\abe_lizbethp\\OneDrive - Iowa State University\\Documents\\ArcGIS\\Projects\\Digital_Acre_SOPs\\Digital_Acre_SOPs.gdb\\Rows_polygon'>

### Number Rows

In [8]:
#Calculate center of row polygon and use latitude to number polygons 
arcpy.env.outputCoordinateSystem = arcpy.Describe("R1_R9_w_Numb").spatialReference

# Set local variables
in_features = "Rows_polygon"
arcpy.management.CalculateGeometryAttributes(in_features, [["Row_Latitude", "CENTROID_Y"]])

#Sort polygons by latitude (descending) and assign numbering 
#Set local variables 
in_dataset = "Rows_polygon"
out_dataset  = "Row_polygon_w_Numb"

#Process: Sort (Sort) (management)
arcpy.management.Sort(in_dataset, out_dataset, sort_field=[["Rep_Numb","ASCENDING"],["Row_Latitude", "DESCENDING"]], spatial_sort_method="UR")

fc = "Row_polygon_w_Numb"
field ="Rep_Numb"
cursor = arcpy.SearchCursor(fc)
rep_list =  []

# Use SearchCursor with list comprehension to return a
# unique set of values in the specified field
for row in cursor:
    #print(row.getValue(field))
    rep = row.getValue(field)
    rep_list.append(rep)
    

# Set local variables
inTable = "Row_polygon_w_Numb"
fieldName = "Row_Numb"
expression = "autoIncrement()"

codeblock = """
rec = 0
m = 0
rep_list = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
def autoIncrement():
    global rec
    global m
    m=m+1
        
    pStart    = 1
    pInterval = 1
    if (rec == 0):
        rec = pStart
    else:
        rec += pInterval
            
    if  rep_list[m]!= rep_list[m-1]:
        rec = 0
    return rec"""
 
# Run AddField
arcpy.management.AddField(inTable, fieldName, "SHORT")
 
# Run CalculateField 
arcpy.management.CalculateField(inTable, fieldName, expression, "PYTHON3", 
                                codeblock)

#All rows 0 need to be changed to 48 


Traceback (most recent call last):
  File "<expression>", line 1, in <module>
  File "<string>", line 17, in autoIncrement
IndexError: list index out of range


ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
Failed to execute (CalculateField).


In [9]:
# Set local variables
inTable = "Row_polygon_w_Numb"
fieldName = "Row_Numb"
expression = "row48(!Row_Numb!)"

codeblock = """
def row48(row):
    if (row == 0):
        numb = 48
    else:
        numb = row
    return numb"""

# Run CalculateField 
arcpy.management.CalculateField(inTable, fieldName, expression, "PYTHON3", 
                                codeblock)



<Result 'Row_polygon_w_Numb'>

### Rep 9 Row 48, overwrite row number to 48 

# Divide Rows into Zone + Numbering

### Manually split rows into zones

In [5]:
#Calculate center of zone polygon and use longitude for number zone
# Set local variables
env.workspace = r"C:\Users\abe_lizbethp\OneDrive - Iowa State University\Documents\2022_Digital_Acre\Tags_Metadata\Tags_Metadata.gdb\Row_Zones_1_9"
in_features = "Row_Zones_1_9"
arcpy.management.CalculateGeometryAttributes(in_features, [["Zone_Longitude", "CENTROID_X"]])

<Result 'Row_Zones_1_9'>

### Numerate Zones

In [7]:
#Sort polygons 
#Set local variables 
in_dataset = "Row_Zones_1_9"
out_dataset  = r"C:\Users\abe_lizbethp\OneDrive - Iowa State University\Documents\ArcGIS\Projects\Digital_Acre_SOPs\Digital_Acre_SOPs.gdb\Zones_w_Numb"

#Process: Sort (Sort) (management)
arcpy.management.Sort(in_dataset, out_dataset, sort_field=[["Rep_Numb","ASCENDING"],["Row_Numb","ASCENDING"],["Zone_Longitude", "ASCENDING"]], spatial_sort_method="UR")

env.workspace = r'C:\Users\abe_lizbethp\OneDrive - Iowa State University\Documents\ArcGIS\Projects\Digital_Acre_SOPs\Digital_Acre_SOPs.gdb'
fc = "Zones_w_Numb"
field ="Row_Numb"
cursor = arcpy.SearchCursor(fc)
rep_list = []

# Use SearchCursor with list comprehension to return a
# unique set of values in the specified field
for row in cursor:
    #print(row.getValue(field))
    rep = row.getValue(field)
    rep_list.append(rep)
    
# Set local variables
inTable = "Zones_w_Numb"
fieldName = "Zone_Numb"
expression = "zone()"

codeblock = """
rec = 0
m = 0
rep_list=[1, 1, 1, 2, 2, 2, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 29, 29, 30, 30, 30, 31, 31, 31, 32, 32, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 36, 36, 36, 37, 37, 37, 38, 38, 38, 39, 39, 39, 40, 40, 40, 41, 41, 41, 42, 42, 42, 43, 43, 43, 44, 44, 44, 45, 45, 45, 46, 46, 46, 47, 47, 47, 48, 48, 48, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 29, 29, 30, 30, 30, 31, 31, 31, 32, 32, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 36, 36, 36, 37, 37, 37, 38, 38, 38, 39, 39, 39, 40, 40, 40, 41, 41, 41, 42, 42, 42, 43, 43, 43, 44, 44, 44, 45, 45, 45, 46, 46, 46, 47, 47, 47, 48, 48, 48, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 29, 29, 30, 30, 30, 31, 31, 31, 32, 32, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 36, 36, 36, 37, 37, 37, 38, 38, 38, 39, 39, 39, 40, 40, 40, 41, 41, 41, 42, 42, 42, 43, 43, 43, 44, 44, 44, 45, 45, 45, 46, 46, 46, 47, 47, 47, 48, 48, 48, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44, 45, 45, 46, 46, 47, 47, 48, 48, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 29, 29, 30, 30, 30, 31, 31, 31, 32, 32, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 36, 36, 36, 37, 37, 37, 38, 38, 38, 39, 39, 39, 40, 40, 40, 41, 41, 41, 42, 42, 42, 43, 43, 43, 44, 44, 44, 45, 45, 45, 46, 46, 46, 47, 47, 47, 48, 48, 48, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 29, 29, 30, 30, 30, 31, 31, 31, 32, 32, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 36, 36, 36, 37, 37, 37, 38, 38, 38, 39, 39, 39, 40, 40, 40, 41, 41, 41, 42, 42, 42, 43, 43, 43, 44, 44, 44, 45, 45, 45, 46, 46, 46, 47, 47, 47, 48, 48, 48, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44, 45, 45, 46, 46, 47, 47, 48, 48, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 29, 29, 30, 30, 30, 31, 31, 31, 32, 32, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 36, 36, 36, 37, 37, 38, 38, 38, 39, 39, 39, 40, 40, 40, 41, 41, 41, 42, 42, 42, 43, 43, 43, 44, 44, 44, 45, 45, 45, 46, 46, 46, 47, 47, 47, 48, 48, 48, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 29, 29, 30, 30, 30, 31, 31, 31, 32, 32, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 36, 36, 36, 37, 37, 37, 38, 38, 38, 39, 39, 39, 40, 40, 40, 41, 41, 41, 42, 42, 42, 43, 43, 43, 44, 44, 44, 45, 45, 45, 46, 46, 46, 47, 47, 47]
def zone():
    global rec
    global m
    
    pInterval = 1
    rec += pInterval
    
    if rep_list[m]!= rep_list[m-1]:
        rec = 1
    m=m+1
    return rec"""
 
# Run AddField
arcpy.management.AddField(inTable, fieldName, "SHORT")
 
# Run CalculateField 
arcpy.management.CalculateField(inTable, fieldName, expression, "PYTHON3", 
                                codeblock)

<Result 'Zones_w_Numb'>

# Join ISU Field Metadata to Sentera Plants

### Merge Sentera Shapefiles 


In [9]:
R1_plants = r"T:\current\Projects\Deere\CPSI\2022\Digital Acre\Sentera Emergence and Per-Plant Measurements\Treatment 1\6equ325_Treatment-1\Day18_Output.shp"
R2_plants = r"T:\current\Projects\Deere\CPSI\2022\Digital Acre\Sentera Emergence and Per-Plant Measurements\Treatment 2\9mpby8z_Treatment-2\Day18_Output.shp"
R3_plants = r"T:\current\Projects\Deere\CPSI\2022\Digital Acre\Sentera Emergence and Per-Plant Measurements\Treatment 3\24xy3ne_Treatment-3\Day18_Output.shp"
R4_plants = r"T:\current\Projects\Deere\CPSI\2022\Digital Acre\Sentera Emergence and Per-Plant Measurements\Treatment 4\7r5p4a2_Treatment-4\Day18_Output.shp"
R5_plants = r"T:\current\Projects\Deere\CPSI\2022\Digital Acre\Sentera Emergence and Per-Plant Measurements\Treatment 5\xple2d8_Treatment-5\Day18_Output.shp"
R6_plants = r"T:\current\Projects\Deere\CPSI\2022\Digital Acre\Sentera Emergence and Per-Plant Measurements\Treatment 6\3wnjrty_Treatment-6\Day18_Output.shp"
R7_plants = r"T:\current\Projects\Deere\CPSI\2022\Digital Acre\Sentera Emergence and Per-Plant Measurements\Treatment 7\3txwg9t_Treatment-7\Day18_Output.shp"
R8_plants = r"T:\current\Projects\Deere\CPSI\2022\Digital Acre\Sentera Emergence and Per-Plant Measurements\Treatment 8\9ju6wap_Treatment-8\Day18_Output.shp"
R9_plants = r"T:\current\Projects\Deere\CPSI\2022\Digital Acre\Sentera Emergence and Per-Plant Measurements\Treatment 9\azuvqp0_2022-DigitalAcre-Treatment9\2022-DigitalAcre-Treatment9\2022-DigitalAcre-Treatment9.shp"


#Local Variables
in_data = [R1_plants, R2_plants, R3_plants, R4_plants, R5_plants, R6_plants, R7_plants, R8_plants, R9_plants]
out_data = "Merge_Sentera_Plants"
#Merge all stakes feature class into a single feature class 
arcpy.Merge_management(in_data, out_data)


<Result 'C:\\Users\\abe_lizbethp\\OneDrive - Iowa State University\\Documents\\ArcGIS\\Projects\\Digital_Acre_SOPs\\Digital_Acre_SOPs.gdb\\Merge_Sentera_Plants'>

### Spatial Join Sentera + ISU Field Metadata

In [11]:
target_features = "Merge_Sentera_Plants"
join_features = "Zones_w_Numb"
out_feature_class = "Sentera_w_Metadata"
join_operation = "JOIN_ONE_TO_ONE"
join_type = "KEEP_ALL"
match_option = "INTERSECT"

arcpy.analysis.SpatialJoin(target_features, join_features, out_feature_class, 
                           join_operation, join_type, "", match_option)

<Result 'C:\\Users\\abe_lizbethp\\OneDrive - Iowa State University\\Documents\\ArcGIS\\Projects\\Digital_Acre_SOPs\\Digital_Acre_SOPs.gdb\\Sentera_w_Metadata'>

# Numbering Plants 

###  Assign Lat and Lon to Sentera Points  

In [10]:
# Set environment settings (Project GDB) 
env.outputCoordinateSystem = arcpy.Describe("Merge_Sentera_Plants").spatialReference

# Set local variables
in_features = "Merge_Sentera_Plants"
arcpy.management.CalculateGeometryAttributes(in_features, [["Latitude", "Point_Y"], ["Longitude", "Point_X"]],"","","","DD")

<Result 'Merge_Sentera_Plants'>

### Read Plants shapefile as dataframe and number plants

In [14]:
# Set local variables
in_table = "Sentera_w_Metadata"
out_xls = r"C:\Users\abe_lizbethp\OneDrive - Iowa State University\Documents\ArcGIS\Projects\Digital_Acre_SOPs\Sentera_w_Metadata.xlsx"

# Run TableToExcel
arcpy.conversion.TableToExcel(in_table, out_xls)

<Result 'C:\\Users\\abe_lizbethp\\OneDrive - Iowa State University\\Documents\\ArcGIS\\Projects\\Digital_Acre_SOPs\\Sentera_w_Metadata.xlsx'>

In [15]:
df = pd.read_excel(out_xls)
df_sort = df.sort_values(by=['Rep_Numb', 'Row_Numb','Zone_Numb', 'Longitude'])
df_sort = df_sort.reset_index()
zone_number = df_sort["Zone_Numb"]

In [16]:
plant_number = []
n = 0
m = 0

for i in zone_number:
    m=m+1
    n = n+1
    plant_number.append(n)
    
    if zone_number[m]!=zone_number[m-1]:
        #print(m)
        n = 0

KeyError: 22339

In [17]:
df_sort['Plant_Number']= plant_number

### Convert df to excel, excel to table, table to XY Display

In [19]:
out_xls = r"C:\Users\abe_lizbethp\OneDrive - Iowa State University\Documents\ArcGIS\Projects\Digital_Acre_SOPs\Sentera_w_Numbers.xlsx"
df_sort.to_excel(out_xls)

In [20]:
Input_Excel_File = out_xls
Output_Table = "Sentera_w_Plant_Numb"
arcpy.conversion.ExcelToTable(Input_Excel_File, Output_Table)

<Result 'C:\\Users\\abe_lizbethp\\OneDrive - Iowa State University\\Documents\\ArcGIS\\Projects\\Digital_Acre_SOPs\\Digital_Acre_SOPs.gdb\\Sentera_w_Plant_Numb'>

In [ ]:
arcpy.management.XYTableToPoint("Sentera_w_Plant_Numb", r"C:\Users\abe_lizbethp\OneDrive - Iowa State University\Documents\ArcGIS\Projects\Digital_Acre_SOPs\Digital_Acre_SOPs.gdb\Sentera_w_Plant_Number_XYTabletoPoint", "Longitude", "Latitude", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')